In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix

### Import dữ liệu sau processed

In [8]:
df = pd.read_csv('Reviews_preprocessed.csv')
df.head(10)

,IDRestaurant,Comment_new,rating_encoded
0,1,chiên sống,1
1,1,ngon,1
2,1,thừa,0
3,1,kêu đói háo_hức thành khóc ròng khóc ròng khóc...,1
4,1,tặng uống,2
5,1,thề lắm tức_giận mừng háo_hức khô chú_ý minion...,0
6,1,thử hơi kén không_ngon_miệng dập tắt nghiện cư...,2
7,1,kết_hợp không_thấy không_cung_cấp,0
8,1,dở ăn_tây tây ngon,1
9,1,cười,0


### Feature Engineering

In [9]:
df.rating_encoded.value_counts()

2    22094
1     4116
0     3707
Name: rating_encoded, dtype: int64

In [10]:
source = df['Comment_new']
source

0                                               chiên sống
1                                                     ngon
2                                                     thừa
3        kêu đói háo_hức thành khóc ròng khóc ròng khóc...
4                                                tặng uống
                               ...                        
29912          ngon skdkdjabgkflagshjxjozn bzokzbzjznb đào
29913                                                  góp
29914    lừa mong rửa_kỉ điện_thoại đừng phá nuôi lừa c...
29915                                         ngon sạch_sẽ
29916                                              dạy đắt
Name: Comment_new, Length: 29917, dtype: object

In [11]:
target = df['rating_encoded']
target

0        1
1        1
2        0
3        1
4        2
        ..
29912    2
29913    2
29914    0
29915    2
29916    0
Name: rating_encoded, Length: 29917, dtype: int64

In [12]:
text_data = np.array(source)
text_data

array(['chiên sống', 'ngon', 'thừa', ...,
       'lừa mong rửa_kỉ điện_thoại đừng phá nuôi lừa chụp hình_ảnh tắt xào cải_ngọt',
       'ngon sạch_sẽ', 'dạy đắt'], dtype=object)

In [13]:
target_data = np.array(target)
target_data

array([1, 1, 0, ..., 0, 2, 0], dtype=int64)

In [14]:
count = CountVectorizer()
count.fit(text_data)
bag_of_words = count.transform(text_data)
bag_of_words

<29917x27609 sparse matrix of type '<class 'numpy.int64'>'
	with 332713 stored elements in Compressed Sparse Row format>

In [15]:
X=bag_of_words.toarray()
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [16]:
X.shape

(29917, 27609)

In [17]:
y = target_data
y

array([1, 1, 0, ..., 0, 2, 0], dtype=int64)

### Split data into train test

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.20)

### Select model : Chọn model Naive Bayes

In [13]:
clf = MultinomialNB()
model = clf.fit(X_train, y_train)

In [14]:
y_pred = clf.predict(X_test)

In [15]:
print('score Scikit learn - train: ', model.score(X_train,y_train))

score Scikit learn - train:  0.844106463878327


In [16]:
print('score Scikit learn: ', model.score(X_test,y_test))

score Scikit learn:  0.7962901069518716


In [17]:
from sklearn.metrics import accuracy_score
print("Accuracy is ", accuracy_score(y_test,y_pred)*100,"%")

Accuracy is  79.62901069518716 %


In [18]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.53      0.60       734
           1       0.42      0.08      0.14       846
           2       0.82      0.98      0.89      4404

    accuracy                           0.80      5984
   macro avg       0.65      0.53      0.54      5984
weighted avg       0.75      0.80      0.75      5984



Nhận xét:
Mô hình  có accuracy tổng thể là 0.80, với các chỉ số precision và recall khác nhau đối với từng lớp. Lớp positive (2) có hiệu suất tốt nhất với precision và recall lần lượt là 0.82 và 0.98, trong khi lớp neutral (1) có hiệu suất thấp nhất với precision và recall với chỉ 0.42 và 0.08. Điều này chứng tỏ rằng mô hình gặp khó khăn trong việc phân loại các đánh giá là neutral.


In [19]:
# Save model
import pickle
filename = 'Naive_Bayes_models_MDS6.pkl'

# Save the model to a file using pickle
with open(filename, 'wb') as file:
    pickle.dump(model, file)



### Người dùng cuối muốn biết rating thuộc nhóm nào dựa vào review?

In [1]:
# Load pickledel from the file
import pickle
with open("Naive_Bayes_models_MDS6.pkl", 'rb') as file:
    loaded_model = pickle.load(file)

In [45]:
# Define feature engineering pipeline 
def feature_engineering_pipeline(text):
    new_text_data = np.append(text_data,text)
    new_bag_of_words = count.transform(new_text_data).toarray()
    last_text_2d = new_bag_of_words[-1].reshape(1, -1)
    rating_predicted = loaded_model.predict(last_text_2d)
    return rating_predicted[0]

In [46]:
# Define rating dictionary:
rating_code = {
    0: "negative",
    1: "neutral",
    2: "positive"
}

In [48]:
# Load data process pipeline:
import Data_processing_pipeline
text = input("Bạn hãy cho bình luận về một nhà hàng nào đó: ")
text_after_process = Data_processing_pipeline.text_processing_pipeline(text)
print("Text after process: ", text_after_process)
print("Rating: ", rating_code[feature_engineering_pipeline(text_after_process)])

Bạn hãy cho bình luận về một nhà hàng nào đó: Nhà hàng này trên cả tuyệt vời. Đồ ăn tươi ngon
Text after process:  tươi ngon
Rating:  positive


### Propose model : Chọn model Support Vector Machine

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

df_cleaned = df.dropna(subset=['Comment_new'])

# Split the data into training and testing sets
X_train_svm, X_test_svm, y_train_svm, y_test_svm = train_test_split(df_cleaned['Comment_new'], df_cleaned['rating_encoded'], test_size=0.2, random_state=42)

tfidf_vectorizer = TfidfVectorizer()

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_svm.astype(str))
X_test_tfidf = tfidf_vectorizer.transform(X_test_svm.astype(str))

svm_classifier = SVC()

# Train the classifier
svm_classifier.fit(X_train_tfidf, y_train_svm)

# Make predictions
y_pred_svm = svm_classifier.predict(X_test_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_test_svm, y_pred_svm)
print("Accuracy:", accuracy)

Accuracy: 0.8158422459893048


In [22]:
print(classification_report(y_test_svm, y_pred_svm))

              precision    recall  f1-score   support

           0       0.66      0.63      0.65       717
           1       0.52      0.05      0.10       811
           2       0.84      0.98      0.91      4456

    accuracy                           0.82      5984
   macro avg       0.67      0.56      0.55      5984
weighted avg       0.78      0.82      0.77      5984



Nhận xét: Mô hình SVM có acc nhiều hơn so với NavieBays 2 %, không có thay đổi đáng kể. Mô hình SVM vẫn gặp khó khăn trong việc phân loại các đánh giá là neutral.

### Handle imbalance

In [23]:
# !pip install imblearn
# !pip install -U imbalanced-learn

In [24]:
import imblearn

print("imbalanced-learn version:", imblearn.__version__)

imbalanced-learn version: 0.12.0


In [25]:
from imblearn.over_sampling import SMOTE


In [26]:
X_resampled, y_resampled = SMOTE().fit_resample(X_train,y_train)

In [27]:
# y_resampled.value_counts()
unique_elements, counts = np.unique(y_resampled, return_counts=True)

# Print the value counts
for label, count in zip(unique_elements, counts):
    print(f"Class {label}: {count} samples")

Class 0: 17690 samples
Class 1: 17690 samples
Class 2: 17690 samples


In [28]:
clf = MultinomialNB()
model = clf.fit(X_resampled, y_resampled)

In [29]:
y_pred=model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 529  106   99]
 [ 205  285  356]
 [ 153  666 3585]]
              precision    recall  f1-score   support

           0       0.60      0.72      0.65       734
           1       0.27      0.34      0.30       846
           2       0.89      0.81      0.85      4404

    accuracy                           0.74      5984
   macro avg       0.58      0.62      0.60      5984
weighted avg       0.76      0.74      0.75      5984



### Nhận xét: Accuracy giảm hơn so với khi chưa resample



1. Tạo dữ liệu giả không chính xác: SMOTE tạo ra các mẫu dữ liệu mới bằng cách nội suy giữa các điểm dữ liệu hiện có. Tuy nhiên, quá trình nội suy này có thể tạo ra các mẫu dữ liệu không chính xác hoặc không đại diện cho tập dữ liệu thực tế. Điều này dẫn đến model học các mẫu sai và giảm độ chính xác khi dự đoán.

2. Vấn đề overfitting: SMOhể dẫn đến overfitting, đặc biệt khi tập dữ liệu ban đặc không cân bằng. Khi SMOTE tạo ra nhiều mẫu dữ liệu cho lớp thiểu số, model có thể học các đặc điểm cụ thể của các mẫu dữ liệu giả này thay vì học các đặc điểm chung của lớp thiểu số. Điều nhể dẫn đến model hoạt động tốt trên tập dữ liệu huấn luyện nhưng kém trên tập dữ liệu thử nghiệm.

3. Tăng nhiễu dữ liệu: S thể làm tăng nhiễu trong tập dữ  hợp. Việc thêm nhiễu vào dữ l thể khiến model khó học các mẫu chính xác và làm giảm độ chính xác.